In [1]:
import os
import numpy as np
import tensorflow as tf

from skimage import exposure
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.layers import Input, Dense, add, Activation, Flatten, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, BatchNormalization
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.client import device_lib

## Download dataset 
https://drive.google.com/drive/u/3/folders/1sHh6NvuKX6RB5OytLwf4kaqfQ9svJNDQ

## Load data

In [2]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")

x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

print(x_train.shape, 'train samples')
print(x_test.shape[0], 'test samples')

(50000, 32, 32, 3) train samples
10000 test samples


In [3]:
# It's a multi-class classification problem 
class_index = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4,
               'dog': 5, 'frog': 6,'horse': 7,'ship': 8, 'truck': 9}
print(np.unique(y_train))

[0 1 2 3 4 5 6 7 8 9]


![image](https://img-blog.csdnimg.cn/20190623084800880.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3lqcDE5ODcxMDEz,size_16,color_FFFFFF,t_70)

## Data preprocess

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to one-hot encoding (keras model requires one-hot label as inputs)
num_classes = 10
print(y_train[0])
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train[0])

[9]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


## Data Augmentation

In [5]:
x_train.shape

(50000, 32, 32, 3)

In [6]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

## Early Stopping

In [7]:
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=10, min_delta=0.01)

## Build model

In [8]:
def residual_block(x,o_filters,increase=False):
        stride = (1,1)
        if increase:
            stride = (2,2)

        o1 = Activation('relu')(BatchNormalization(momentum=0.9, epsilon=1e-5)(x))
        conv_1 = Conv2D(o_filters,kernel_size=(3,3),strides=stride,padding='same',
                        kernel_initializer="he_normal",
                        kernel_regularizer=regularizers.l2(weight_decay))(o1)
        o2  = Activation('relu')(BatchNormalization(momentum=0.9, epsilon=1e-5)(conv_1))
        conv_2 = Conv2D(o_filters,kernel_size=(3,3),strides=(1,1),padding='same',
                        kernel_initializer="he_normal",
                        kernel_regularizer=regularizers.l2(weight_decay))(o2)
        if increase:
            projection = Conv2D(o_filters,kernel_size=(1,1),strides=(2,2),padding='same',
                                kernel_initializer="he_normal",
                                kernel_regularizer=regularizers.l2(weight_decay))(o1)
            block = add([conv_2, projection])
        else:
            block = add([conv_2, x])
        return block

In [9]:
stack_n = 5
layers = 6 * stack_n + 2
num_classes = 10
img_rows, img_cols = 32, 32
img_channels = 3
batch_size = 32
epochs = 200
weight_decay = 1e-4
classes_num = 10

img_input = Input(shape=(x_train.shape[1:]))

x = Conv2D(filters=16,kernel_size=(3,3),strides=(1,1),
           padding='same',
           kernel_initializer="he_normal",
           kernel_regularizer=regularizers.l2(weight_decay))(img_input)

# input: 32x32x16 output: 32x32x16
for _ in range(stack_n):
    x = residual_block(x, 16, False)

# input: 32x32x16 output: 16x16x32
x = residual_block(x, 32, True)
for _ in range(1, stack_n):
    x = residual_block(x, 32, False)

# input: 16x16x32 output: 8x8x64
x = residual_block(x, 64, True)
for _ in range(1, stack_n):
    x = residual_block(x, 64, False)

x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
x = Activation('relu')(x)
x = GlobalAveragePooling2D()(x)

# input: 64 output: 10
x = Dense(classes_num,
          activation='softmax',
          kernel_initializer="he_normal",
          kernel_regularizer=regularizers.l2(weight_decay))(x)

model = Model(img_input, x)

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 16)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

## Set Optimizer

In [11]:
opt = keras.optimizers.SGD(learning_rate=0.005, momentum=0.9, nesterov=True)

## Training

In [12]:
# Compile the model with loss function and optimizer, and evaluate with accuracy
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
    epochs=epochs,
    validation_data=(x_test, y_test),
    callbacks=[es])

/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/200
1563/1563 [==============================] - 388s 246ms/step - loss: 1.8316 - accuracy: 0.4133 - val_loss: 2.0146 - val_accuracy: 0.4318
Epoch 2/200
1563/1563 [==============================] - 385s 246ms/step - loss: 1.4406 - accuracy: 0.5657 - val_loss: 1.4284 - val_accuracy: 0.5788
Epoch 3/200
1563/1563 [==============================] - 385s 246ms/step - loss: 1.2654 - accuracy: 0.6365 - val_loss: 1.2325 - val_accuracy: 0.6539
Epoch 4/200
1563/1563 [==============================] - 385s 247ms/step - loss: 1.1502 - accuracy: 0.6776 - val_loss: 1.2074 - val_accuracy: 0.6776
Epoch 5/200
1563/1563 [==============================] - 384s 246ms/step - loss: 1.0607 - accuracy: 0.7098 - val_loss: 0.9667 - val_accuracy: 0.7444
Epoch 6/200
1563/1563 [==============================] - 384s 245ms/step - loss: 0.9849 - accuracy: 0.7359 - val_loss: 0.9854 - val_accuracy: 0.7395
Epoch 7/200
1563/1563 [==============================] - 383s 245ms/step - loss: 0.9333 - accuracy: 0.7562

In [13]:
y_pred = model.predict(x_test)
print(y_pred.shape) # 10000 samples, each sample with probaility of 10 classes

(10000, 10)


In [14]:
y_pred[0] 

array([4.5249693e-09, 9.9839371e-01, 7.0752058e-08, 1.5730620e-06,
       1.4878971e-08, 5.7997919e-08, 2.3007878e-09, 1.1426085e-09,
       7.4821305e-09, 1.6045875e-03], dtype=float32)

In [15]:
np.argmax(y_pred[0]) # argmax to find the predict class with highest probability. 9=truck

1

In [16]:
y_pred = np.argmax(y_pred, axis=1)

## DO NOT MODIFY CODE BELOW!
**Please screen shot your results and post it on your report**

In [17]:
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

In [18]:
assert y_pred.shape == (10000, )

In [19]:
y_test = np.load("y_test.npy")
print("Accuracy of my model on test set: ", accuracy_score(y_test, y_pred))

Accuracy of my model on test set:  0.8749
